In [3]:
import numpy as np
import hapke
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import linregress
from scipy import optimize
from matplotlib.collections import PatchCollection
from matplotlib.patches import Patch
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
import matplotlib.image as mpimg
from mpl_toolkits import mplot3d
from scipy.interpolate import interp1d
from pyvims import VIMS
from pyvims.misc import MAPS
import random
from scipy import stats

matplotlib.use('Qt5Agg')
plt.rcParams.update({'font.size': 12})

colors = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']

def first_degree(w, a, b):
    return a + b * w

def second_degree(w, a1, b1, c1):
    return a1 + b1 * w + c1 * w ** 2

In [77]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as patches
x = 0
y = 1
# Define the two points D THETA M
inktomi = np.array([123,62, 0.12])
inktomi_err = np.array([42,15, 0.06])

creusa = np.array([96, 65,0.14])
creusa_err = np.array([96*0.375, 65*0.21,0.14*0.45])

t_rhea = np.array([54,46,0.36])
t_rhea_err = np.array([20,8,0.04])

t_dione = np.array([10,83, 0.31])
t_dione_err = np.array([10,2, 0.03])

t_enceladus = np.array([60, 62,0.12])
t_enceladus_err = np.array([12,4,0.03])

l_rhea = np.array([90,70,0.10])
l_rhea_err = np.array([9,1,0.02])

l_dione = np.array([67,71,0.08])
l_dione_err = np.array([7,2,0.01])

l_enceladus = np.array([101,54, 0.11])
l_enceladus_err = np.array([5,2, 0.01])

r_enc = np.array([103,54, 0.18])
r_enc_err = np.array([103*0.33,54*0.11, 0.18*0.55])

r_dione = np.array([62,65, 0.18])
r_dione_err = np.array([62*0.5,65*0.12, 0.18*0.4])


# Create a figure and axis
fig, ax = plt.subplots(figsize=(5,4))

# Plot the points
ax.errorbar(inktomi[x], inktomi[y],xerr=inktomi_err[x],yerr=inktomi_err[y], color = colors[0],marker='o')
ax.errorbar(creusa[x], creusa[y],xerr=creusa_err[x],yerr=creusa_err[y], color = colors[1],marker='o')

ax.errorbar(t_rhea[x], t_rhea[y],xerr=t_rhea_err[x],yerr=t_rhea_err[y], color = colors[0],marker='x')
ax.errorbar(t_dione[x], t_dione[y],xerr=t_dione_err[x],yerr=t_dione_err[y], color = colors[1],marker='x')
#ax.errorbar(t_enceladus[x], t_enceladus[y],xerr=t_enceladus_err[x],yerr=t_enceladus_err[y], color = colors[2],marker='x')

ax.errorbar(l_rhea[x], l_rhea[y],xerr=l_rhea_err[x],yerr=l_rhea_err[y], color = colors[0],marker='^')
ax.errorbar(l_dione[x], l_dione[y],xerr=l_rhea_err[x],yerr=l_rhea_err[y], color = colors[1],marker='^')
#ax.errorbar(l_enceladus[x], l_enceladus[y],xerr=l_enceladus_err[x],yerr=l_enceladus_err[y], color = colors[2],marker='^')

ax.errorbar(r_enc[x], r_enc[y],xerr=r_enc_err[x],yerr=r_enc_err[y], color = colors[2],marker='s')
ax.errorbar(r_dione[x], r_dione[y],xerr=r_dione_err[x],yerr=r_dione_err[y], color = colors[1],marker='s')

#ax.annotate('E-ring', xy=(75,40), xytext=(0.6,0.4),
#            arrowprops={'arrowstyle': '->', 'lw': 3, 'color': 'blue'},
#            va='center')

e_ring1 = (100,40)
e_ring2 = (25,75)

e_ring = patches.FancyArrowPatch(e_ring1, e_ring2,
                                arrowstyle='->',
                                lw=3,
                                mutation_scale=15,
                                color='blue')
ax.add_patch(e_ring)

plasma1 = (100,45)
plasma2 = (25,80)

plasma = patches.FancyArrowPatch(plasma1, plasma2,
                                arrowstyle='->',
                                lw=3,
                                mutation_scale=15,
                                color='red')
ax.add_patch(plasma)

impact1 = (25,57)
impact2 = (125,57)

impact = patches.FancyArrowPatch(impact1, impact2,
                                arrowstyle='->',
                                lw=3,
                                mutation_scale=15,
                                color='grey')
ax.add_patch(impact)



text = 'Arrow Text'
text_x = (inktomi[x] + creusa[y]) / 2
text_y = (inktomi[x] + creusa[y]) / 2
#ax.text(text_x, text_y, text, ha='center', va='center', color='black')

# Set axis labels
ax.set_xlabel(r'$D$ [$\mu$m]')
ax.set_ylabel(r'$\bar{\theta}$  [deg]')
plt.grid()
# Show the plot
plt.show()

KeyboardInterrupt: 

In [82]:
x = 0
y = 2

# Create a figure and axis
fig, ax = plt.subplots(figsize=(5,4))

# Plot the points
ax.errorbar(inktomi[x], inktomi[y],xerr=inktomi_err[x],yerr=inktomi_err[y], color = colors[0],marker='o')
ax.errorbar(creusa[x], creusa[y],xerr=creusa_err[x],yerr=creusa_err[y], color = colors[1],marker='o')

ax.errorbar(t_rhea[x], t_rhea[y],xerr=t_rhea_err[x],yerr=t_rhea_err[y], color = colors[0],marker='x')
ax.errorbar(t_dione[x], t_dione[y],xerr=t_dione_err[x],yerr=t_dione_err[y], color = colors[1],marker='x')
ax.errorbar(t_enceladus[x], t_enceladus[y],xerr=t_enceladus_err[x],yerr=t_enceladus_err[y], color = colors[2],marker='x')

ax.errorbar(l_rhea[x], l_rhea[y],xerr=l_rhea_err[x],yerr=l_rhea_err[y], color = colors[0],marker='^')
ax.errorbar(l_dione[x], l_dione[y],xerr=l_rhea_err[x],yerr=l_rhea_err[y], color = colors[1],marker='^')
ax.errorbar(l_enceladus[x], l_enceladus[y],xerr=l_enceladus_err[x],yerr=l_enceladus_err[y], color = colors[2],marker='^')

ax.errorbar(r_enc[x], r_enc[y],xerr=r_enc_err[x],yerr=r_enc_err[y], color = colors[2],marker='s')
ax.errorbar(r_dione[x], r_dione[y],xerr=r_dione_err[x],yerr=r_dione_err[y], color = colors[2],marker='s')

e_ring1 = (100,0.3)
e_ring2 = (25,0.1)

e_ring = patches.FancyArrowPatch(e_ring1, e_ring2,
                                arrowstyle='->',
                                lw=3,
                                mutation_scale=15,
                                color='blue')
ax.add_patch(e_ring)

plasma1 = (100,0.12)
plasma2 = (25,0.32)

plasma = patches.FancyArrowPatch(plasma1, plasma2,
                                arrowstyle='->',
                                lw=3,
                                mutation_scale=15,
                                color='red')
ax.add_patch(plasma)

impact1 = (40,0.1)
impact2 = (115,0.3)

impact = patches.FancyArrowPatch(impact1, impact2,
                                arrowstyle='->',
                                lw=3,
                                mutation_scale=15,
                                color='grey')
ax.add_patch(impact)

text = 'Arrow Text'
text_x = (inktomi[x] + creusa[y]) / 2
text_y = (inktomi[x] + creusa[y]) / 2
#ax.text(text_x, text_y, text, ha='center', va='center', color='black')

# Set axis labels
ax.set_xlabel(r'$D$ [$\mu$m]')
ax.set_ylabel(r'$m$ [-]')
plt.grid()
# Show the plot
plt.show()

KeyboardInterrupt: 

In [84]:
x = 1
y = 2

# Create a figure and axis
fig, ax = plt.subplots(figsize=(5,4))

# Plot the points
ax.errorbar(inktomi[x], inktomi[y],xerr=inktomi_err[x],yerr=inktomi_err[y], color = colors[0],marker='o')
ax.errorbar(creusa[x], creusa[y],xerr=creusa_err[x],yerr=creusa_err[y], color = colors[1],marker='o')

ax.errorbar(t_rhea[x], t_rhea[y],xerr=t_rhea_err[x],yerr=t_rhea_err[y], color = colors[0],marker='x')
ax.errorbar(t_dione[x], t_dione[y],xerr=t_dione_err[x],yerr=t_dione_err[y], color = colors[1],marker='x')
ax.errorbar(t_enceladus[x], t_enceladus[y],xerr=t_enceladus_err[x],yerr=t_enceladus_err[y], color = colors[2],marker='x')

ax.errorbar(l_rhea[x], l_rhea[y],xerr=l_rhea_err[x],yerr=l_rhea_err[y], color = colors[0],marker='^')
ax.errorbar(l_dione[x], l_dione[y],xerr=l_rhea_err[x],yerr=l_rhea_err[y], color = colors[1],marker='^')
ax.errorbar(l_enceladus[x], l_enceladus[y],xerr=l_enceladus_err[x],yerr=l_enceladus_err[y], color = colors[2],marker='^')

ax.errorbar(r_enc[x], r_enc[y],xerr=r_enc_err[x],yerr=r_enc_err[y], color = colors[2],marker='s')
ax.errorbar(r_dione[x], r_dione[y],xerr=r_dione_err[x],yerr=r_dione_err[y], color = colors[2],marker='s')

#ax.annotate('E-ring', xy=(75,40), xytext=(0.6,0.4),
#            arrowprops={'arrowstyle': '->', 'lw': 3, 'color': 'blue'},
#            va='center')

e_ring1 = (40,0.35)
e_ring2 = (80,0.1)

e_ring = patches.FancyArrowPatch(e_ring1, e_ring2,
                                arrowstyle='->',
                                lw=3,
                                mutation_scale=15,
                                color='blue')
ax.add_patch(e_ring)

plasma1 = (40,0.1)
plasma2 = (80,0.35)

plasma = patches.FancyArrowPatch(plasma1, plasma2,
                                arrowstyle='->',
                                lw=3,
                                mutation_scale=15,
                                color='red')
ax.add_patch(plasma)

impact1 = (57,0.1)
impact2 = (57,0.35)

impact = patches.FancyArrowPatch(impact1, impact2,
                                arrowstyle='->',
                                lw=3,
                                mutation_scale=15,
                                color='grey')
ax.add_patch(impact)

text = 'Arrow Text'
text_x = (inktomi[x] + creusa[y]) / 2
text_y = (inktomi[x] + creusa[y]) / 2
#ax.text(text_x, text_y, text, ha='center', va='center', color='black')

# Set axis labels
ax.set_xlabel(r'$\bar{\theta}$ [deg]')
ax.set_ylabel(r'$m$ [-]')
plt.grid()
# Show the plot
plt.show()

KeyboardInterrupt: 

In [ ]:
fig, ax = plt.subplots(figsize=(5,4))

# Plot the points
#plt.axvline(x=1, color=colors[0], linestyle='-', label='Rhea')
#plt.axvline(x=1, color=colors[1], linestyle='-', label='Dione')
#plt.axvline(x=1, color=colors[2], linestyle='-', label='Enceladus')

#ax.errorbar(inktomi[x], inktomi[y],xerr=inktomi_err[x],yerr=inktomi_err[y], color = 'grey',marker='o',label='Recent impacts')
#ax.errorbar(t_rhea[x], t_rhea[y],xerr=t_rhea_err[x],yerr=t_rhea_err[y], color = 'grey',marker='x',label='Trailing hemispheres')
#ax.errorbar(l_rhea[x], l_rhea[y],xerr=l_rhea_err[x],yerr=l_rhea_err[y], color = 'grey',marker='^',label='Leading hemispheres')
#ax.errorbar(r_enc[x], r_enc[y],xerr=r_enc_err[x],yerr=r_enc_err[y], color = 'grey',marker='s',label='Ridges')

#ax.annotate('E-ring', xy=(75,40), xytext=(0.6,0.4),
#            arrowprops={'arrowstyle': '->', 'lw': 3, 'color': 'blue'},
#            va='center')


e_ring = patches.FancyArrowPatch(e_ring1, e_ring2,
                                arrowstyle='->',
                                lw=3,
                                mutation_scale=15,
                                color='blue',label='E-ring deposition')
ax.add_patch(e_ring)

plasma1 = (40,0.1)
plasma2 = (80,0.35)

plasma = patches.FancyArrowPatch(plasma1, plasma2,
                                arrowstyle='->',
                                lw=3,
                                mutation_scale=15,
                                color='red',label='Plasma')
ax.add_patch(plasma)

impact1 = (57,0.1)
impact2 = (57,0.35)

impact = patches.FancyArrowPatch(impact1, impact2,
                                arrowstyle='->',
                                lw=3,
                                mutation_scale=15,
                                color='grey',label='Impact')
ax.add_patch(impact)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left',title='Arrows')
# Show the plot
plt.show()

In [13]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.proj3d import proj_transform
from mpl_toolkits.mplot3d.axes3d import Axes3D
from matplotlib.text import Annotation
class Annotation3D(Annotation):

    def __init__(self, text, xyz, *args, **kwargs):
        super().__init__(text, xy=(0, 0), *args, **kwargs)
        self._xyz = xyz

    def draw(self, renderer):
        x2, y2, z2 = proj_transform(*self._xyz, self.axes.M)
        self.xy = (x2, y2)
        super().draw(renderer)

def _annotate3D(ax, text, xyz, *args, **kwargs):
    '''Add anotation `text` to an `Axes3d` instance.'''

    annotation = Annotation3D(text, xyz, *args, **kwargs)
    ax.add_artist(annotation)

setattr(Axes3D, 'annotate3D', _annotate3D)

In [28]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.set_xlabel(r'$m$')
ax.set_ylabel(r'$\bar{\theta}$')
ax.set_zlabel(r'$D$')


# inktomi
ax.scatter([0.12], [60], [120], s=30, marker='o', color=colors[0], label = 'Inktomi')
# creusa
ax.scatter([0.14], [65], [90], s=30, marker='x', color=colors[0], label = 'Creusa')

ax.annotate3D('Recent craters', (0.12, 60, 120),
              xytext=(30, -30),
              textcoords='offset points',
              bbox=dict(boxstyle="round", fc="lightyellow"))
# trailing rhea
ax.scatter([0.35], [50], [50], s=30, marker='o', color=colors[1], label = 'Rhea Trailing')

# trailing dione
ax.scatter([0.31], [80], [20], s=30, marker='x', color=colors[1], label = 'Dione Trailing')

ax.annotate3D('Trailing', (0.35, 60, 40),
              xytext=(30, -30),
              textcoords='offset points',
              bbox=dict(boxstyle="round", fc="lightyellow"))

# leading rhea
ax.scatter([0.1], [70], [80], s=30, marker='o', color=colors[2], label = 'Rhea Leading')

# leading dione
ax.scatter([0.08], [72], [65], s=30, marker='x', color=colors[2], label = 'Dione Leading')

ax.annotate3D('Leading', (0.1, 70, 70),
              xytext=(30, -30),
              textcoords='offset points',
              bbox=dict(boxstyle="round", fc="lightyellow"))


#ax.annotate3D('Recent craters', (0.12, 60, 120), xytext=(3, 3), textcoords='offset points')
#ax.annotate3D('point 2', (0, 1, 0),
#              xytext=(-30, -30),
#              textcoords='offset points',
#              arrowprops=dict(ec='black', fc='white', shrink=2.5))
#ax.annotate3D('Recent craters', (0.12, 60, 120),
#              xytext=(30, -30),
#              textcoords='offset points',
#              bbox=dict(boxstyle="round", fc="lightyellow"),
#              arrowprops=dict(arrowstyle="-|>", ec='black', fc='white', lw=5))
ax.set_title('')
#fig.legend(loc='upper right', bbox_to_anchor=(1.2, 1))
fig.tight_layout()
plt.show()

KeyboardInterrupt: 

In [2]:
T = 100
N = 20

opt_c = hapke.opticalconstants(T,sensitivity = N)
n_c = opt_c['n']
k_c = opt_c['k']
wav_c = opt_c['wav']

opt_a = hapke.opticalconstants(T, sensitivity= N,crystallinity=False)
n_a = opt_a['n']
k_a = opt_a['k']
wav_a = opt_a['wav']

int_opt = hapke.inter_optical_constants(wav_c, wav_a, n_c, k_c)

wav = np.array(int_opt['wav'])
n_c = int_opt['n']
k_c = int_opt['k']

opt_constants = [n_c,k_c,n_a,k_a]

In [40]:
cube1 = VIMS('1702386165_1')
cube2 = VIMS('1507736136_1')

pixel1 = []
pixel2 = []

for i in range(cube1.NS):
    for j in range(cube1.NL):
        pix = cube1[i+1,j+1]
        #print([hapke.is_micron_dip(pix), not pix.limb, np.mean(np.array(pix.spectrum)) > 0.025, pix.inc < 90, pix.phase > 5, pix.eme < 90])
        #if pix is not None and not hapke.is_micron_dip(pix) and not pix.limb and np.mean(np.array(pix.spectrum)) > 0.025 and pix.inc < 90 and pix.phase > 5 and pix.eme < 90:

        if pix is not None and not pix.limb and np.mean(np.array(pix.spectrum)) > 0.025 and pix.inc < 90 and pix.phase > 5 and pix.eme < 90:

            pixel1.append(cube1[i+1,j+1])

for i in range(cube2.NS):
    for j in range(cube2.NL):
        pix = cube2[i+1,j+1]
        #print([hapke.is_micron_dip(pix), not pix.limb, np.mean(np.array(pix.spectrum)) > 0.025, pix.inc < 90, pix.phase > 5, pix.eme < 90])
        #if pix is not None and not hapke.is_micron_dip(pix) and not pix.limb and np.mean(np.array(pix.spectrum)) > 0.025 and pix.inc < 90 and pix.phase > 5 and pix.eme < 90:

        if pix is not None and not pix.limb and np.mean(np.array(pix.spectrum)) > 0.025 and pix.inc < 90 and pix.phase > 5 and pix.eme < 90:

            pixel2.append(cube2[i+1,j+1])


In [41]:
loc_21 = []
loc_22 = []

a=0

for pix in pixel1:
    IF = pix.spectrum
    wav1 = pix.wvlns

    loc_21.append(hapke.cryst_area(IF,wav1)['loc_2'][0])

for pix in pixel2:
    IF = pix.spectrum
    wav1 = pix.wvlns

    loc_22.append(hapke.cryst_area(IF,wav1)['loc_2'][0])

In [44]:
p_loc1 = []
loc_new1 = []

p_loc2 = []
loc_new2 = []

print('yes')

for p in range(len(pixel1)):

    if 2.024 < loc_21[p] < 2.0295:
        p_loc1.append(pixel1[p])
        loc_new1.append(loc_21[p])

for p in range(len(pixel2)):

    if 2.024 < loc_22[p] < 2.0295:
        p_loc2.append(pixel2[p])
        loc_new2.append(loc_22[p])

yes


In [ ]:
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
from matplotlib.patches import Polygon

bg = MAPS['DIONE']
fig, (ax1,ax2) = bg.subplots(1, 2, figsize=(20, 10))

value = loc_21

cmap = plt.get_cmap('viridis')  # You can choose any other colormap
norm = Normalize(vmin=2.024, vmax=2.0295)

patches = []

for p in range(len(p_loc1)):

    corners_lon = p_loc1[p].corners.lonlat[0, :]
    corners_lat = p_loc1[p].corners.lonlat[1, :]

    vertices = [(corners_lon[0], corners_lat[0]), (corners_lon[1], corners_lat[1]),(corners_lon[2], corners_lat[2]),(corners_lon[3], corners_lat[3])]

    polygon = Polygon(vertices, closed=True, alpha=0.6)
    color = cmap(norm(value[p]))
    polygon.set_facecolor(color)

    patches.append(polygon)

pc = PatchCollection(patches, match_original=True)
ax1.add_collection(pc)

sm = ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])  # Set an empty array to the ScalarMappable
cbar_ax = fig.add_axes([0.45, 0.57, 0.02, 0.4])  # Adjust position as needed
cbar = fig.colorbar(sm, cax=cbar_ax)
cbar.set_label(r'Loc 2-$\mu$m')



value = loc_22

cmap = plt.get_cmap('viridis')  # You can choose any other colormap
norm = Normalize(vmin=2.024, vmax=2.0295)

patches = []

for p in range(len(p_loc2)):

    corners_lon = p_loc2[p].corners.lonlat[0, :]
    corners_lat = p_loc2[p].corners.lonlat[1, :]

    vertices = [(corners_lon[0], corners_lat[0]), (corners_lon[1], corners_lat[1]),(corners_lon[2], corners_lat[2]),(corners_lon[3], corners_lat[3])]

    polygon = Polygon(vertices, closed=True, alpha=0.6)
    color = cmap(norm(value[p]))
    polygon.set_facecolor(color)

    patches.append(polygon)

pc = PatchCollection(patches, match_original=True)
ax2.add_collection(pc)
# Create a custom colorbar for ax2
sm2 = ScalarMappable(cmap=cmap, norm=norm)
sm2.set_array([])  # Set an empty array to the ScalarMappable
cbar_ax2 = fig.add_axes([0.85, 0.57, 0.02, 0.4])  # Adjust position as needed
cbar2 = fig.colorbar(sm2, cax=cbar_ax2)
cbar2.set_label(r'Loc 2-$\mu$m')


print('yes')

plt.show()

yes


C:\Users\USUARIO\AppData\Local\Temp\ipykernel_4684\1497716506.py:71: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.show()


In [ ]:
bg = MAPS['ENC_SP']
fig, ax = bg.figure(figsize=(4,4))
plt.show()

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_13364\1472073021.py:3: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations.
  plt.show()


In [52]:
cube = VIMS('1829241298_1')

pixel = []

for i in range(cube.NS):
    for j in range(cube.NL):
        pix = cube[i+1,j+1]
        #print([hapke.is_micron_dip(pix), not pix.limb, np.mean(np.array(pix.spectrum)) > 0.025, pix.inc < 90, pix.phase > 5, pix.eme < 90])
        if pix is not None and hapke.is_micron_dip(pix) and not pix.limb and np.mean(np.array(pix.spectrum)) > 0.025 and pix.inc < 90 and pix.phase > 5 and pix.eme < 90:

        #if pix is not None and not pix.limb and np.mean(np.array(pix.spectrum)) > 0.025 and pix.inc < 90 and pix.phase > 5 and pix.eme < 90:

            pixel.append(cube[i+1,j+1])

In [53]:
loc_2 = []
loc_2_err = []
area_1 = []
fits = {}

D_fit = []
D_err = []
theta_fit = []
theta_err = []

m = []
err_m = []

a=0

for pix in pixel:
    IF = pix.spectrum
    wav1 = pix.wvlns

    angles = [np.deg2rad(pix.eme),np.deg2rad(pix.inc),np.deg2rad(pix.phase)]

    random_fit = hapke.random_fit(1,wav, angles, IF, wav1,n_c,k_c,n_a,k_a)

    errors = hapke.print_error_correlation(random_fit['fit'])

    D_err.append(errors[1]*1000000)
    theta_err.append(errors[0])

    D_fit.append(random_fit['fit'].x[1]*1000000)
    theta_fit.append(random_fit['fit'].x[0])

    IF_c = hapke.hapke_model_mixed([random_fit['fit'].x[0],random_fit['fit'].x[1]],wav,angles,n_c,k_c,n_a,k_a)['IF']

    area_1.append(hapke.cryst_area_one_peak(IF,wav1)['area']/hapke.cryst_area_one_peak(IF_c,wav)['area'])

    loc_2.append(hapke.cryst_area(IF,wav1)['loc_2'][0])
    loc_2_err.append(hapke.cryst_area(IF,wav1)['loc_2'][1])

    m.append(hapke.loc_to_m(hapke.cryst_area(IF,wav1)['loc_2'][0],hapke.cryst_area(IF,wav1)['loc_2'][1])[0])
    err_m.append(hapke.loc_to_m(hapke.cryst_area(IF,wav1)['loc_2'][0],hapke.cryst_area(IF,wav1)['loc_2'][1])[1])


    a+=1
    print(str(a) + '/' + str(len(pixel)))


[1.3662847751397567]
[0.0, 0.0, 0.0]


C:\Users\USUARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\optimize\_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


1/846
[1.327889783991607]
[0.0, 0.0, 0.0]
2/846
[1.3147402786349316]
[0.0, 0.0, 0.0]
3/846
[1.3138640562134334]
[0.0, 0.0, 0.0]
4/846
[1.3171081643078955]
[0.0, 0.0, 0.0]
5/846
[1.347766568632984]
[0.0, 0.0, 0.0]
6/846
[1.3054563611420789]
[0.0, 0.0, 0.0]
7/846
[1.3248109895194597]
[0.0, 0.0, 0.0]
8/846
[1.3029122481685702]
[0.0, 0.0, 0.0]
9/846
[1.3021379720884698]
[0.0, 0.0, 0.0]
10/846
[1.2739272199747453]
[0.0, 0.0, 0.0]
11/846
[1.2781023144406907]
[0.0, 0.0, 0.0]
12/846
[1.3296157603265564]
[0.0, 0.0, 0.0]
13/846
[1.2492790810030985]
[0.0, 0.0, 0.0]
14/846
[1.270511973131757]
[0.0, 0.0, 0.0]
15/846
[1.2982719638328009]
[0.0, 0.0, 0.0]
16/846
[1.2319378869146465]
[0.0, 0.0, 0.0]
17/846
[1.3402628942807806]
[0.0, 0.0, 0.0]
18/846
[1.3376446656354677]
[0.0, 0.0, 0.0]
19/846
[1.2700631969906262]
[0.0, 0.0, 0.0]
20/846
[1.238248744746413]
[0.0, 0.0, 0.0]
21/846
[1.2630207075201165]
[0.0, 0.0, 0.0]
22/846
[1.274232092155242]
[0.0, 0.0, 0.0]
23/846
[1.1392994890912693]
[0.0, 0.0, 0.0]
24

In [68]:
p_new = []
D_new = []
D_err_new = []

theta_new= []
theta_err_new = []

p_loc = []
loc_new = []
loc_err_new = []
m_err_new = []
m_new = []

p_area = []
area_new= []

print('yes')
print('yes')
for p in range(len(pixel)):
    if 60 < D_fit[p] < 150 and np.deg2rad(70) > theta_fit[p] > np.deg2rad(00):
       p_new.append(pixel[p])
       D_new.append(D_fit[p])
       D_err_new.append(D_err[p])
       theta_new.append(np.rad2deg(theta_fit[p]))
       theta_err_new.append(np.rad2deg(theta_err[p]))

    if 2.015 < loc_2[p] < 2.028:

        if err_m[p]/m[p] < 0.8 and m[p] < 0.270:
            p_loc.append(pixel[p])
            loc_new.append(loc_2[p])
            loc_err_new.append(loc_2_err[p])
            m_new.append(m[p])
            m_err_new.append(err_m[p])

    if 1.5 > area_1[p] > 00.65:
        p_area.append(pixel[p])
        area_new.append(area_1[p])

yes
yes


In [69]:
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
from matplotlib.patches import Polygon

bg = MAPS['ENCELADUS']


value_list = [np.array(D_new),np.array(theta_new),np.array(m_new),np.array(area_new),np.array(D_err_new)/np.array(D_new),np.array(theta_err_new)/np.array(theta_new),np.array(m_err_new)/np.array(m_new)]

label_list = [r'$D$ [$\mu$m]',r'$\bar{\theta}$ [deg]',r'$m$ [-]',r'Area [-]',r'$\Delta D/D$',r'$\Delta \bar{\theta}/\bar{\theta}$',r'$\Delta m/m$']

p_list = [p_new,p_new,p_loc,p_area,p_new,p_new,p_loc]



for INDEX in range(len(value_list)):

    fig, ax1 = bg.subplots(figsize=(5.5,4))

    value = value_list[INDEX]

    cmap = plt.get_cmap('viridis')  # You can choose any other colormap
    norm = Normalize(vmin=min(value), vmax=max(value))

    patches = []

    for p in range(len(p_list[INDEX])):

        corners_lon = p_list[INDEX][p].corners.lonlat[0, :]
        corners_lat = p_list[INDEX][p].corners.lonlat[1, :]

        vertices = [(corners_lon[0], corners_lat[0]), (corners_lon[1], corners_lat[1]),(corners_lon[2], corners_lat[2]),(corners_lon[3], corners_lat[3])]

        polygon = Polygon(vertices, closed=True, alpha=0.6)
        color = cmap(norm(value[p]))
        polygon.set_facecolor(color)

        patches.append(polygon)

    pc = PatchCollection(patches, match_original=True)
    ax1.add_collection(pc)

    sm = ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])  # Set an empty array to the ScalarMappable
    #cbar_ax = fig.add_axes([0.95, 0.1, 0.04, 0.7])  # Adjust position as needed cax=cbar_ax
    cbar = fig.colorbar(sm)
    cbar.set_label(label_list[INDEX])

plt.show()

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_2264\3278157133.py:46: MatplotlibDeprecationWarning: Unable to determine Axes to steal space for Colorbar. Using gca(), but will raise in the future. Either provide the *cax* argument to use as the Axes for the Colorbar, provide the *ax* argument to steal space from it, or add *mappable* to an Axes.
  cbar = fig.colorbar(sm)

KeyboardInterrupt



In [20]:
# PARAMETERS MAP

from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
from matplotlib.patches import Polygon

bg = MAPS['RHEA']
fig, ((ax1,ax2),(ax3,ax4)) = bg.subplots(2, 2)

value = np.array(D_new)

cmap = plt.get_cmap('viridis')  # You can choose any other colormap
norm = Normalize(vmin=min(value), vmax=max(value))

patches = []

for p in range(len(p_new)):

    corners_lon = p_new[p].corners.lonlat[0, :]
    corners_lat = p_new[p].corners.lonlat[1, :]

    vertices = [(corners_lon[0], corners_lat[0]), (corners_lon[1], corners_lat[1]),(corners_lon[2], corners_lat[2]),(corners_lon[3], corners_lat[3])]

    polygon = Polygon(vertices, closed=True, alpha=0.6)
    color = cmap(norm(value[p]))
    polygon.set_facecolor(color)

    patches.append(polygon)

pc = PatchCollection(patches, match_original=True)
ax1.add_collection(pc)

sm = ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])  # Set an empty array to the ScalarMappable
cbar_ax = fig.add_axes([0.45, 0.57, 0.02, 0.4])  # Adjust position as needed
cbar = fig.colorbar(sm, cax=cbar_ax)
cbar.set_label(r'$D$ [$\mu$m]')


value = np.array(theta_new)

cmap = plt.get_cmap('viridis')  # You can choose any other colormap
norm = Normalize(vmin=min(value), vmax=max(value))

patches = []

for p in range(len(p_new)):

    corners_lon = p_new[p].corners.lonlat[0, :]
    corners_lat = p_new[p].corners.lonlat[1, :]

    vertices = [(corners_lon[0], corners_lat[0]), (corners_lon[1], corners_lat[1]),(corners_lon[2], corners_lat[2]),(corners_lon[3], corners_lat[3])]

    polygon = Polygon(vertices, closed=True, alpha=0.6)
    color = cmap(norm(value[p]))
    polygon.set_facecolor(color)

    patches.append(polygon)

pc = PatchCollection(patches, match_original=True)
ax2.add_collection(pc)
# Create a custom colorbar for ax2
sm2 = ScalarMappable(cmap=cmap, norm=norm)
sm2.set_array([])  # Set an empty array to the ScalarMappable
cbar_ax2 = fig.add_axes([0.85, 0.57, 0.02, 0.4])  # Adjust position as needed
cbar2 = fig.colorbar(sm2, cax=cbar_ax2)
cbar2.set_label(r'$\bar{\theta}$ [deg]')

m = []
for i in range(len(m_new)):
    m.append(hapke.loc_to_m(loc_new[i],0)[0])
value = np.array(m)

cmap = plt.get_cmap('viridis')  # You can choose any other colormap
norm = Normalize(vmin=min(value), vmax=max(value))

patches = []

for p in range(len(p_loc)):

    corners_lon = p_loc[p].corners.lonlat[0, :]
    corners_lat = p_loc[p].corners.lonlat[1, :]

    vertices = [(corners_lon[0], corners_lat[0]), (corners_lon[1], corners_lat[1]),(corners_lon[2], corners_lat[2]),(corners_lon[3], corners_lat[3])]

    polygon = Polygon(vertices, closed=True, alpha=0.6)
    color = cmap(norm(value[p]))
    polygon.set_facecolor(color)

    patches.append(polygon)

pc = PatchCollection(patches, match_original=True)
ax3.add_collection(pc)

# Create a custom colorbar for ax2
sm3 = ScalarMappable(cmap=cmap, norm=norm)
sm3.set_array([])  # Set an empty array to the ScalarMappable
cbar_ax3 = fig.add_axes([0.45, 0.065, 0.02, 0.4])  # Adjust position as needed
cbar3 = fig.colorbar(sm3, cax=cbar_ax3)
cbar3.set_label(r'$m$ [-]')


value = area_new

cmap = plt.get_cmap('viridis')  # You can choose any other colormap
norm = Normalize(vmin=min(value), vmax=max(value))

patches = []

for p in range(len(p_area)):

    corners_lon = p_area[p].corners.lonlat[0, :]
    corners_lat = p_area[p].corners.lonlat[1, :]

    vertices = [(corners_lon[0], corners_lat[0]), (corners_lon[1], corners_lat[1]),(corners_lon[2], corners_lat[2]),(corners_lon[3], corners_lat[3])]

    polygon = Polygon(vertices, closed=True, alpha=0.6)
    color = cmap(norm(value[p]))
    polygon.set_facecolor(color)

    patches.append(polygon)

pc = PatchCollection(patches, match_original=True)
ax4.add_collection(pc)
# Create a custom colorbar for ax2 [0.45, 0.55, 0.02, 0.4]
sm4 = ScalarMappable(cmap=cmap, norm=norm)
sm4.set_array([])  # Set an empty array to the ScalarMappable [0.45, 0.55, 0.02, 0.4]
cbar_ax4 = fig.add_axes([0.85, 0.065, 0.02, 0.4])  # Adjust position as needed
cbar4 = fig.colorbar(sm4, cax=cbar_ax4)
cbar4.set_label(r'Area [-]')

print('yes')

plt.show()

yes


C:\Users\USUARIO\AppData\Local\Temp\ipykernel_17672\59921644.py:134: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.show()


KeyboardInterrupt: 

In [22]:
# RELATIVE ERROR MAP

from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
from matplotlib.patches import Polygon

bg = MAPS['RHEA']
fig, ((ax1,ax2),(ax3,ax4)) = bg.subplots(2, 2, figsize=(20, 10))

value = np.array(D_err_new) / np.array(D_new)

cmap = plt.get_cmap('viridis')  # You can choose any other colormap
norm = Normalize(vmin=min(value), vmax=max(value))

patches = []

for p in range(len(p_new)):

    corners_lon = p_new[p].corners.lonlat[0, :]
    corners_lat = p_new[p].corners.lonlat[1, :]

    vertices = [(corners_lon[0], corners_lat[0]), (corners_lon[1], corners_lat[1]),(corners_lon[2], corners_lat[2]),(corners_lon[3], corners_lat[3])]

    polygon = Polygon(vertices, closed=True, alpha=0.6)
    color = cmap(norm(value[p]))
    polygon.set_facecolor(color)

    patches.append(polygon)

pc = PatchCollection(patches, match_original=True)
ax1.add_collection(pc)

sm = ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])  # Set an empty array to the ScalarMappable
cbar_ax = fig.add_axes([0.45, 0.57, 0.02, 0.4])  # Adjust position as needed
cbar = fig.colorbar(sm, cax=cbar_ax)
cbar.set_label(r'$\Delta D/D$')


value = np.array(theta_err_new) / np.array(theta_new)

cmap = plt.get_cmap('viridis')  # You can choose any other colormap
norm = Normalize(vmin=min(value), vmax=max(value))

patches = []

for p in range(len(p_new)):

    corners_lon = p_new[p].corners.lonlat[0, :]
    corners_lat = p_new[p].corners.lonlat[1, :]

    vertices = [(corners_lon[0], corners_lat[0]), (corners_lon[1], corners_lat[1]),(corners_lon[2], corners_lat[2]),(corners_lon[3], corners_lat[3])]

    polygon = Polygon(vertices, closed=True, alpha=0.6)
    color = cmap(norm(value[p]))
    polygon.set_facecolor(color)

    patches.append(polygon)

pc = PatchCollection(patches, match_original=True)
ax2.add_collection(pc)
# Create a custom colorbar for ax2
sm2 = ScalarMappable(cmap=cmap, norm=norm)
sm2.set_array([])  # Set an empty array to the ScalarMappable
cbar_ax2 = fig.add_axes([0.85, 0.57, 0.02, 0.4])  # Adjust position as needed
cbar2 = fig.colorbar(sm2, cax=cbar_ax2)
cbar2.set_label(r'$\Delta \bar{\theta} / \bar{\theta}$')


value = np.array(m_err_new) / np.array(m_new)

cmap = plt.get_cmap('viridis')  # You can choose any other colormap
norm = Normalize(vmin=min(value), vmax=max(value))

patches = []

for p in range(len(p_loc)):

    corners_lon = p_loc[p].corners.lonlat[0, :]
    corners_lat = p_loc[p].corners.lonlat[1, :]

    vertices = [(corners_lon[0], corners_lat[0]), (corners_lon[1], corners_lat[1]),(corners_lon[2], corners_lat[2]),(corners_lon[3], corners_lat[3])]

    polygon = Polygon(vertices, closed=True, alpha=0.6)
    color = cmap(norm(value[p]))
    polygon.set_facecolor(color)

    patches.append(polygon)

pc = PatchCollection(patches, match_original=True)
ax3.add_collection(pc)

# Create a custom colorbar for ax2
sm3 = ScalarMappable(cmap=cmap, norm=norm)
sm3.set_array([])  # Set an empty array to the ScalarMappable
cbar_ax3 = fig.add_axes([0.45, 0.065, 0.02, 0.4])  # Adjust position as needed
cbar3 = fig.colorbar(sm3, cax=cbar_ax3)
cbar3.set_label(r'$\Delta m/m$')


value = area_new

cmap = plt.get_cmap('viridis')  # You can choose any other colormap
norm = Normalize(vmin=min(value), vmax=max(value))

patches = []

for p in range(len(p_area)):

    corners_lon = p_area[p].corners.lonlat[0, :]
    corners_lat = p_area[p].corners.lonlat[1, :]

    vertices = [(corners_lon[0], corners_lat[0]), (corners_lon[1], corners_lat[1]),(corners_lon[2], corners_lat[2]),(corners_lon[3], corners_lat[3])]

    polygon = Polygon(vertices, closed=True, alpha=0.6)
    color = cmap(norm(value[p]))
    polygon.set_facecolor(color)

    patches.append(polygon)

pc = PatchCollection(patches, match_original=True)
ax4.add_collection(pc)
# Create a custom colorbar for ax2 [0.45, 0.55, 0.02, 0.4]
sm4 = ScalarMappable(cmap=cmap, norm=norm)
sm4.set_array([])  # Set an empty array to the ScalarMappable [0.45, 0.55, 0.02, 0.4]
cbar_ax4 = fig.add_axes([0.85, 0.065, 0.02, 0.4])  # Adjust position as needed
cbar4 = fig.colorbar(sm4, cax=cbar_ax4)
cbar4.set_label(r'Area [-]')

print('yes')

plt.show()

yes


KeyboardInterrupt: 

In [16]:
fig, ax = plt.subplots()

for p in p_loc:
    ax.plot(p.wvlns,p.spectrum)
plt.show()

KeyboardInterrupt: 

In [14]:
from matplotlib.patches import Polygon

value = np.array(loc_err_new) /( np.array(loc_new)-2)

cmap = plt.get_cmap('viridis')  # You can choose any other colormap
norm = Normalize(vmin=min(value), vmax=max(value))

bg = MAPS['ENC_SP']
fig, ax = bg.figure(figsize=(20, 10))

patches = []

for p in range(len(p_loc)):

    corners_lon = p_loc[p].corners.lonlat[0, :]
    corners_lat = p_loc[p].corners.lonlat[1, :]

    for j in range(len(corners_lon)):
        if corners_lon[j] > 180:
            corners_lon[j] += -360

    vertices = [(corners_lon[0], corners_lat[0]), (corners_lon[1], corners_lat[1]),(corners_lon[2], corners_lat[2]),(corners_lon[3], corners_lat[3])]

    polygon = Polygon(vertices, closed=True, alpha=0.6)
    color = cmap(norm(value[p]))
    polygon.set_facecolor(color)

    patches.append(polygon)

pc = PatchCollection(patches, match_original=True)
ax.add_collection(pc)
sm = ScalarMappable(cmap=cmap, norm=norm)
cbar = plt.colorbar(sm)
cbar.set_label(r'$\bar{\theta}$ [deg]')

plt.show()

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_14468\3170619423.py:33: MatplotlibDeprecationWarning: Unable to determine Axes to steal space for Colorbar. Using gca(), but will raise in the future. Either provide the *cax* argument to use as the Axes for the Colorbar, provide the *ax* argument to steal space from it, or add *mappable* to an Axes.
  cbar = plt.colorbar(sm)


KeyboardInterrupt: 

In [ ]:
fig, ax = plt.subplots()

for pix in pixel:
    ax.plot(pix.wvlns,pix.spectrum)
plt.show()

In [ ]:
p_new = []
D_new = []
theta_new= []
p_loc = []
loc_new = []
p_area = []
area_new= []
print('yes')
print('yes')
for p in range(len(pixel)):
    if 20 < D_fit[p] < 500 and 1.4 > theta_fit[p] > 0.7:
        p_new.append(pixel[p])
        D_new.append(D_fit[p])
        theta_new.append(np.rad2deg(theta_fit[p]))

    if 2.015 < loc_2[p] < 2.05:
        p_loc.append(pixel[p])
        loc_new.append(loc_2[p])

    if 1.6 > area_1[p] > 0.1:
        p_area.append(pixel[p])
        area_new.append(area_1[p])

In [ ]:
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
from matplotlib.patches import Polygon

bg = MAPS[cube.target_name]
fig, ((ax1,ax2),(ax3,ax4)) = bg.subplots(2, 2, figsize=(20, 10))

value = D_new

cmap = plt.get_cmap('viridis')  # You can choose any other colormap
norm = Normalize(vmin=min(value), vmax=max(value))

patches = []

for p in range(len(p_new)):

    corners_lon = p_new[p].corners.lonlat[0, :]
    corners_lat = p_new[p].corners.lonlat[1, :]

    vertices = [(corners_lon[0], corners_lat[0]), (corners_lon[1], corners_lat[1]),(corners_lon[2], corners_lat[2]),(corners_lon[3], corners_lat[3])]

    polygon = Polygon(vertices, closed=True, alpha=0.6)
    color = cmap(norm(value[p]))
    polygon.set_facecolor(color)

    patches.append(polygon)

pc = PatchCollection(patches, match_original=True)
ax1.add_collection(pc)

sm = ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])  # Set an empty array to the ScalarMappable
cbar_ax = fig.add_axes([0.45, 0.57, 0.02, 0.4])  # Adjust position as needed
cbar = fig.colorbar(sm, cax=cbar_ax)
cbar.set_label(r'$D$ [$\mu$m]')



value = theta_new

cmap = plt.get_cmap('viridis')  # You can choose any other colormap
norm = Normalize(vmin=min(value), vmax=max(value))

patches = []

for p in range(len(p_new)):

    corners_lon = p_new[p].corners.lonlat[0, :]
    corners_lat = p_new[p].corners.lonlat[1, :]

    vertices = [(corners_lon[0], corners_lat[0]), (corners_lon[1], corners_lat[1]),(corners_lon[2], corners_lat[2]),(corners_lon[3], corners_lat[3])]

    polygon = Polygon(vertices, closed=True, alpha=0.6)
    color = cmap(norm(value[p]))
    polygon.set_facecolor(color)

    patches.append(polygon)

pc = PatchCollection(patches, match_original=True)
ax2.add_collection(pc)
# Create a custom colorbar for ax2
sm2 = ScalarMappable(cmap=cmap, norm=norm)
sm2.set_array([])  # Set an empty array to the ScalarMappable
cbar_ax2 = fig.add_axes([0.85, 0.57, 0.02, 0.4])  # Adjust position as needed
cbar2 = fig.colorbar(sm2, cax=cbar_ax2)
cbar2.set_label(r'$\bar{\theta} [deg]$')



value = loc_new

cmap = plt.get_cmap('viridis')  # You can choose any other colormap
norm = Normalize(vmin=min(value), vmax=max(value))

patches = []

for p in range(len(p_loc)):

    corners_lon = p_loc[p].corners.lonlat[0, :]
    corners_lat = p_loc[p].corners.lonlat[1, :]

    vertices = [(corners_lon[0], corners_lat[0]), (corners_lon[1], corners_lat[1]),(corners_lon[2], corners_lat[2]),(corners_lon[3], corners_lat[3])]

    polygon = Polygon(vertices, closed=True, alpha=0.6)
    color = cmap(norm(value[p]))
    polygon.set_facecolor(color)

    patches.append(polygon)

pc = PatchCollection(patches, match_original=True)
ax3.add_collection(pc)

# Create a custom colorbar for ax2
sm3 = ScalarMappable(cmap=cmap, norm=norm)
sm3.set_array([])  # Set an empty array to the ScalarMappable
cbar_ax3 = fig.add_axes([0.45, 0.065, 0.02, 0.4])  # Adjust position as needed
cbar3 = fig.colorbar(sm3, cax=cbar_ax3)
cbar3.set_label(r'Loc 2-$\mu$m')


value = area_new

cmap = plt.get_cmap('viridis')  # You can choose any other colormap
norm = Normalize(vmin=min(value), vmax=max(value))

patches = []

for p in range(len(p_area)):

    corners_lon = p_area[p].corners.lonlat[0, :]
    corners_lat = p_area[p].corners.lonlat[1, :]

    vertices = [(corners_lon[0], corners_lat[0]), (corners_lon[1], corners_lat[1]),(corners_lon[2], corners_lat[2]),(corners_lon[3], corners_lat[3])]

    polygon = Polygon(vertices, closed=True, alpha=0.6)
    color = cmap(norm(value[p]))
    polygon.set_facecolor(color)

    patches.append(polygon)

pc = PatchCollection(patches, match_original=True)
ax4.add_collection(pc)
# Create a custom colorbar for ax2 [0.45, 0.55, 0.02, 0.4]
sm4 = ScalarMappable(cmap=cmap, norm=norm)
sm4.set_array([])  # Set an empty array to the ScalarMappable [0.45, 0.55, 0.02, 0.4]
cbar_ax4 = fig.add_axes([0.85, 0.065, 0.02, 0.4])  # Adjust position as needed
cbar4 = fig.colorbar(sm4, cax=cbar_ax4)
cbar4.set_label(r'Area [-]')

print('yes')

In [ ]:
plt.show()